In [1]:
import sys
import os
from elasticsearch import Elasticsearch
from zipfile import ZipFile
import re
import pandas as pd
import json
import pytrec_eval
import requests

In [2]:
def evaluate_run(run_path, qrel_path):
    run = pd.read_csv(run_path, sep = ' ', names = ['topic','Q0','id','rank','score','team'])

    #run['topic_mustbe'] = 0

    #i = 1
    #t = 0
    #for idx, row in run.iterrows():
        #if row['rank']>1:
            #run.at[idx,'topic_mustbe']=t
        #else:
            #t+=1
            #run.at[idx,'topic_mustbe']=t

#'touche2020-task2-relevance-withbaseline(1).qrels'
    qrel = pd.read_csv(qrel_path, sep = ' ', names = ['topic','Q0','id','relevance'])

    qrels = {}
    for i in range(1,51):
        qrels[str(i)] = {}
    
    for idx, row in qrel.iterrows():
        qrels[str(row['topic'])][row['id']] = row['relevance']
    
    runs = {}
    for i in range(1,51):
        runs[str(i)] = {}
    
    for idx, row in run.iterrows():
        runs[str(row['topic'])][row['id']] = row['score']

    evaluator = pytrec_eval.RelevanceEvaluator(
        qrels,{'map_cut', 'ndcg_cut', 'recall', 'P'})
#{'map_cut', 'ndcg_cut', 'recall'}
    res = evaluator.evaluate(runs)

    results = {}
    recall = {}
    pres = {}
    for key in res:
        results[key] = res[key]['ndcg_cut_10']
        recall[key] = res[key]['recall_10']
        pres[key] = res[key]['P_10']
        #print(res[key])
    
    filename = re.split('\.', sys.argv[1])
    with open((filename[0]+'_evaluated.json'), 'w') as f:
        json.dump(results, f)
        
    s = sum(results.values())/50
    r = sum(recall.values())/50
    p = sum(pres.values())/50
    if r+p != 0:
        f1 = (2*(r*p)/(r+p))
    else: 
        f1 = 0
    '''
    print("Average ndcg_cut_10: ", s)
    print("Average recall_10: ", r)
    print("Av. precision: ", p)
    print("F1-score: ",f1)
    '''    
    return s, r, p, f1


In [9]:
run = []
ndcg = []
rec = []
prec = []
f1s = []

for filename in os.listdir('results'):
    if filename.endswith(".txt"):
        print(filename)
        s, r, p, f1 = evaluate_run(("results/{}").format(filename), 'touche2020-task2-relevance-withbaseline(1).qrels')
        run.append(filename)
        ndcg.append(s)
        rec.append(r)
        prec.append(p)
        f1s.append(f1)

run_test_d_t_lem.txt
run_test_a_lem.txt
run_test_d_t_lem_70.txt
run_test_t.txt
run_lem_full.txt
run_p_t_a_lem.txt
run_test_t_a_lem.txt
run_test_d_a_lem.txt
run_lem.txt
run_test_t_a_lem_syn.txt
run_t.txt
run_p_t.txt
run_test_t_lem.txt
run_doc.txt
run_test_d_lem.txt
run_p_t_lem.txt
run_p_t_lem_100.txt
run_test_d_t_a_lem.txt
run.txt


In [10]:
df = pd.DataFrame({'run':run,'ndcg_cut10':ndcg, 'recall_10':rec, 'precision':prec, 'f1-score':f1s})
print(df)

                         run  ndcg_cut10  recall_10  precision  f1-score
0       run_test_d_t_lem.txt    0.442337   0.271286      0.424  0.330871
1         run_test_a_lem.txt    0.267192   0.179525      0.262  0.213059
2    run_test_d_t_lem_70.txt    0.360613   0.224717      0.336  0.269315
3             run_test_t.txt    0.348324   0.208376      0.344  0.259538
4           run_lem_full.txt    0.160281   0.088970      0.160  0.114353
5          run_p_t_a_lem.txt    0.295004   0.170775      0.276  0.210996
6       run_test_t_a_lem.txt    0.386302   0.250989      0.374  0.300389
7       run_test_d_a_lem.txt    0.285701   0.189667      0.278  0.225492
8                run_lem.txt    0.000000   0.000000      0.000  0.000000
9   run_test_t_a_lem_syn.txt    0.384752   0.252236      0.376  0.301927
10                 run_t.txt    0.001395   0.000952      0.002  0.001290
11               run_p_t.txt    0.179548   0.110698      0.194  0.140962
12        run_test_t_lem.txt    0.372117   0.232918

In [11]:
df.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
0,run_test_d_t_lem.txt,0.442337,0.271286,0.424,0.330871
6,run_test_t_a_lem.txt,0.386302,0.250989,0.374,0.300389
9,run_test_t_a_lem_syn.txt,0.384752,0.252236,0.376,0.301927
17,run_test_d_t_a_lem.txt,0.384614,0.236975,0.362,0.286439
12,run_test_t_lem.txt,0.372117,0.232918,0.360,0.282840
2,run_test_d_t_lem_70.txt,0.360613,0.224717,0.336,0.269315
3,run_test_t.txt,0.348324,0.208376,0.344,0.259538
14,run_test_d_lem.txt,0.317627,0.195354,0.296,0.235369
5,run_p_t_a_lem.txt,0.295004,0.170775,0.276,0.210996
7,run_test_d_a_lem.txt,0.285701,0.189667,0.278,0.225492


In [12]:
df.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
0,run_test_d_t_lem.txt,0.442337,0.271286,0.424,0.330871
9,run_test_t_a_lem_syn.txt,0.384752,0.252236,0.376,0.301927
6,run_test_t_a_lem.txt,0.386302,0.250989,0.374,0.300389
17,run_test_d_t_a_lem.txt,0.384614,0.236975,0.362,0.286439
12,run_test_t_lem.txt,0.372117,0.232918,0.360,0.282840
2,run_test_d_t_lem_70.txt,0.360613,0.224717,0.336,0.269315
3,run_test_t.txt,0.348324,0.208376,0.344,0.259538
14,run_test_d_lem.txt,0.317627,0.195354,0.296,0.235369
7,run_test_d_a_lem.txt,0.285701,0.189667,0.278,0.225492
1,run_test_a_lem.txt,0.267192,0.179525,0.262,0.213059


In [13]:
df.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
0,run_test_d_t_lem.txt,0.442337,0.271286,0.424,0.330871
9,run_test_t_a_lem_syn.txt,0.384752,0.252236,0.376,0.301927
6,run_test_t_a_lem.txt,0.386302,0.250989,0.374,0.300389
17,run_test_d_t_a_lem.txt,0.384614,0.236975,0.362,0.286439
12,run_test_t_lem.txt,0.372117,0.232918,0.360,0.282840
3,run_test_t.txt,0.348324,0.208376,0.344,0.259538
2,run_test_d_t_lem_70.txt,0.360613,0.224717,0.336,0.269315
14,run_test_d_lem.txt,0.317627,0.195354,0.296,0.235369
7,run_test_d_a_lem.txt,0.285701,0.189667,0.278,0.225492
5,run_p_t_a_lem.txt,0.295004,0.170775,0.276,0.210996


In [14]:
df.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
0,run_test_d_t_lem.txt,0.442337,0.271286,0.424,0.330871
9,run_test_t_a_lem_syn.txt,0.384752,0.252236,0.376,0.301927
6,run_test_t_a_lem.txt,0.386302,0.250989,0.374,0.300389
17,run_test_d_t_a_lem.txt,0.384614,0.236975,0.362,0.286439
12,run_test_t_lem.txt,0.372117,0.232918,0.360,0.282840
2,run_test_d_t_lem_70.txt,0.360613,0.224717,0.336,0.269315
3,run_test_t.txt,0.348324,0.208376,0.344,0.259538
14,run_test_d_lem.txt,0.317627,0.195354,0.296,0.235369
7,run_test_d_a_lem.txt,0.285701,0.189667,0.278,0.225492
1,run_test_a_lem.txt,0.267192,0.179525,0.262,0.213059


In [16]:
run = []
ndcg = []
rec = []
prec = []
f1s = []

for filename in os.listdir('res'):
    if filename.endswith(".txt"):
        print(filename)
        s, r, p, f1 = evaluate_run(("res/{}").format(filename), 'touche2020-task2-relevance-withbaseline(1).qrels')
        run.append(filename)
        ndcg.append(s)
        rec.append(r)
        prec.append(p)
        f1s.append(f1)

In [17]:
df = pd.DataFrame({'run':run,'ndcg_cut10':ndcg, 'recall_10':rec, 'precision':prec, 'f1-score':f1s})

In [18]:
df.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
